In [1]:
import numpy as np
from testCases_v4a import *

In [2]:
def initialize_parameters_deep(layer_dims):
    #layer_dims=[3,4,5],3 unit in first layer, 4 unit in 2nd layer and so on. layer dim also has input X
    np.random.seed(3)
    L=len(layer_dims)
    parameters={}
    for l in range(1,L):
        parameters['W'+str(l)]=np.random.randn((layer_dims[l]),(layer_dims[l-1]))*0.01
        parameters['b'+str(l)]=np.zeros(((layer_dims[l]),1))
    return parameters
    

In [3]:
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


In [4]:
L=len(parameters)/2
print(L)


2.0


In [5]:
def linear_forward(A, W, b):
    #A=Activation fro  previous layer
    Z=np.dot(W,A)+b
    cache=(A,W,b)
    return Z,cache
#cahe as A,W,b

In [6]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


In [7]:
def sigmoid(Z):
    A=1/(1+np.exp(-Z))
    return A,Z
def relu(Z):
    A=np.maximum(0,Z)
    return A,Z

In [8]:
def linear_activation_forward(A_prev, W, b, activation):
    if activation=='sigmoid':
        Z, linear_cache=linear_forward(A_prev, W, b)
        A,activation_cache=sigmoid(Z)
    if activation=='relu':
        Z, linear_cache=linear_forward(A_prev, W, b)
        A,activation_cache=relu(Z)
    cache= (linear_cache,activation_cache)
    return A,cache

#linear cache has  A_prev,W,b
#activation cache has Z
#this both will be combined to become linear activation cache
#like for first layer , it has A_prev,W1,b1,Z1

In [9]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


In [10]:
def L_model_forward(X, parameters):
    caches=[]
    A=X
    L=len(parameters)//2#we used floor decimal so that we dont get error in for loop without // L=2.0 with // L=2
    
    for l in range(1,L):
        A_prev=A
        A,cache=linear_activation_forward(A_prev, parameters['W'+str(l)], parameters['b'+str(l)], activation = "relu")
        caches.append(cache)
        
    AL,cache=linear_activation_forward(A, parameters['W'+str(L)], parameters['b'+str(L)], activation = "sigmoid")
    #in above step we used A isntead of A_prev , beacuse A is latest activation after L-1 computation
    caches.append(cache)
    return AL,caches
#caches has A_prev,W1,b1,Z1 for all layers till(L)
#each index of caches has parameters used for that layer
# the caches tupe consist of two elelmets 1- linear caches, 2-activation cache
        
    

In [11]:
X, parameters = L_model_forward_test_case_2hidden()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))
#caches[2]


AL = [[0.03921668 0.70498921 0.19734387 0.04728177]]
Length of caches list = 3


In [12]:
def compute_cost(AL, Y):
    m = Y.shape[1]#size of array
    cost = (-1/m)*np.sum((Y*np.log(AL))+(1-Y)*np.log(1-AL))
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    return cost


In [13]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

cost = 0.2797765635793422


In [14]:
def sigmoid_backward(dA, cache):
    Z=cache
    s=1/(1+np.exp(-Z))
    dZ=dA*s*(1-s)
    return dZ
def relu_backward(dA, cache):
    Z=cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object
    dZ[Z<=0]=0
    return dZ

In [23]:
#backward propgation
def linear_backward(dZ, cache):
    A_prev,W,b=cache
    m=A_prev.shape[1]
    dW=(1/m)*np.dot(dZ,A_prev.T)
    db=np.sum(dZ,axis=1,keepdims=True)/m
    dA_prev=np.dot(W.T,dZ)
    return dA_prev,dW,db


In [24]:
# Set up some test inputs
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[-1.15171336  0.06718465 -0.3204696   2.09812712]
 [ 0.60345879 -3.72508701  5.81700741 -3.84326836]
 [-0.4319552  -1.30987417  1.72354705  0.05070578]
 [-0.38981415  0.60811244 -1.25938424  1.47191593]
 [-2.52214926  2.67882552 -0.67947465  1.48119548]]
dW = [[ 0.07313866 -0.0976715  -0.87585828  0.73763362  0.00785716]
 [ 0.85508818  0.37530413 -0.59912655  0.71278189 -0.58931808]
 [ 0.97913304 -0.24376494 -0.08839671  0.55151192 -0.10290907]]
db = [[-0.14713786]
 [-0.11313155]
 [-0.13209101]]


In [25]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation=='relu':
        dZ=relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    elif activation=='sigmoid':
        dZ=sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        

        
    return dA_prev, dW, db  
'''
caches-linear cache,activation cache
linear cache=A_prev,W,b
activation cache=Z
'''

'\ncaches-linear cache,activation cache\nlinear cache=A_prev,W,b\nactivation cache=Z\n'

In [26]:
dAL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


In [27]:
def L_model_backward(AL, Y, caches):
    
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    dAL=-(np.divide(Y,AL)-np.divide((1-Y),(1-AL)))
    
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, 'sigmoid')
    print(grads)
    for l in reversed (range(L-1)):#if L=3,hen l=2, the first elemt as per loop will be 1
        current_cache = caches[l]
        grads["dA" + str(l)], grads["dW" + str(l+1)], grads["db" + str(l+1)] = linear_activation_backward(grads["dA" + str(l+1)], current_cache, 'relu')
        #dA1,dw2,db2=(da2,cache[1])
 
    return grads
    
    
    

In [28]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print_grads(grads)
#print(grads)

{'dA1': array([[ 0.12913162, -0.44014127],
       [-0.14175655,  0.48317296],
       [ 0.01663708, -0.05670698]]), 'dW2': array([[-0.39202432, -0.13325855, -0.04601089]]), 'db2': array([[0.15187861]])}
dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
dA1 = [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]]


In [31]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    ### START CODE HERE ### (≈ 3 lines of code)
    for l in range(L):
        parameters["W" + str(l + 1)] = parameters["W" + str(l + 1)] - learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l + 1)] = parameters["b" + str(l + 1)] - learning_rate * grads["db" + str(l + 1)]
    ### END CODE HERE ###
    return parameters

In [32]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]
